In [105]:
import torch

## 框架真正厉害的点？
可以帮我们把反向传播全部计算好

In [106]:
# Method 1 - recommended
x = torch.randn(3,4,requires_grad=True)
x

tensor([[-0.6056, -0.1987,  1.2353, -0.8875],
        [ 0.8571, -1.8104, -0.6508,  0.2827],
        [-1.3963,  1.0834,  1.4556,  2.0931]], requires_grad=True)

In [107]:
# Method 2
x = torch.randn(3,4)
x.requires_grad = True
x

tensor([[ 1.0173,  0.1515, -0.7902, -0.2386],
        [ 1.3049, -0.0285, -0.4235, -1.7789],
        [ 1.1662, -1.9275, -0.8774, -0.6667]], requires_grad=True)

In [108]:
b = torch.randn(3,4,requires_grad=True)
b

tensor([[-0.0529,  1.0883,  0.1806,  1.0977],
        [ 0.0970, -0.6448,  2.5271,  1.1046],
        [-1.0558,  0.2420, -1.5023, -0.4641]], requires_grad=True)

In [109]:
t = x+b
t

tensor([[ 0.9644,  1.2398, -0.6096,  0.8592],
        [ 1.4019, -0.6733,  2.1036, -0.6743],
        [ 0.1105, -1.6855, -2.3797, -1.1308]], grad_fn=<AddBackward0>)

In [110]:
y = t.sum()
y

tensor(-0.4738, grad_fn=<SumBackward0>)

In [111]:
y.backward()
y

tensor(-0.4738, grad_fn=<SumBackward0>)

In [112]:
b.grad
## 这儿的变量就是b，一元一次函数求导

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

虽然没有指定t的required_grad但是需要用到它，也会默认的

In [113]:
x.requires_grad,b.requires_grad,t.requires_grad

(True, True, True)

In [114]:
# calculate flow - example 
x = torch.rand(1)
y = torch.rand(1)
w = torch.rand(1,requires_grad=True)
y = w*x
z = y+b

In [115]:
# check autograde
x.requires_grad,b.requires_grad,w.requires_grad

(False, True, True)

In [116]:
## again
x = torch.rand(1)
b = torch.rand(1,requires_grad=True)
w = torch.rand(1,requires_grad=True)
y = w*x
z = y+b

In [117]:
# check autograde
x.requires_grad,b.requires_grad,w.requires_grad,y.requires_grad # y is needed

(False, True, True, True)

In [118]:
x.is_leaf,w.is_leaf,b.is_leaf,y.is_leaf,z.is_leaf
# z = wx+b,是否为叶子结点

(True, True, True, False, False)

In [119]:
# back propagation
z.backward(retain_graph=True) # 注意，如果不清空梯度会累加起来


In [120]:
w.grad

tensor([0.6309])

In [121]:
b.grad

tensor([1.])

线性回归试水

In [122]:
# 构造一组输入数据X与对应的标签Y
import numpy as np
x_values = [i for i in range(11)]
x_train  = np.array(x_values,dtype = np.float32)
x_train = x_train.reshape(-1,1)
x_train.shape

(11, 1)

In [123]:
import numpy as np
y_values = [2*i+1 for i in range(11)]
y_train = np.array(y_values,dtype=np.float32)
y_train = y_train.reshape(-1,1)
y_train.shape

(11, 1)

In [124]:
import torch 
import torch.nn as nn

线性回归模型

In [125]:
# 线性回归就是不添加激活函数的全连接层
class LinearRegressionModel(nn.Module): # 定义了类的名字为LinearRegressionModel,继承了父类nn.Module
    def __init__(self,input_dim,output_dim): # 实例self从父类继承属性input_dim,output_dim
        super(LinearRegressionModel,self).__init__() # super,可以简单地理解为执行父类的__init__函数，
        # 也就是初始化父类，这样才能访问父类的成员
        self.linear = nn.Linear(input_dim,output_dim)
        
    def forward(self,x):
        out = self.linear(x)
        return out

In [126]:
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim,output_dim,)

In [127]:
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [128]:
# 制定好参数和损失函数
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
criterion = nn.MSELoss()
# 一般分类任务是交叉熵，回归任务mse，也就是mean square error

In [129]:
# 训练模型
for epoch in range(epochs):
    epoch+=1
    # 注意转为tensor
    inputs = torch.tensor(x_train)
    labels = torch.tensor(y_train)
    
    # 梯度要清零每一次迭代
    optimizer.zero_grad()
    
    # 前向传播
    outputs = model(inputs)
    
    # 计算损失
    loss = criterion(outputs,labels)
    
    # 返回传播
    loss.backward()
    
    # 更新权重
    optimizer.step()
    
    if epoch % 50 == 0:
        print("epoch {},loss {}".format(epoch,loss.item()))
    
    
    

epoch 50,loss 0.32141411304473877
epoch 100,loss 0.18332244455814362
epoch 150,loss 0.10456030070781708
epoch 200,loss 0.05963733419775963
epoch 250,loss 0.03401493653655052
epoch 300,loss 0.019400890916585922
epoch 350,loss 0.011065570637583733
epoch 400,loss 0.006311323028057814
epoch 450,loss 0.0035997682716697454
epoch 500,loss 0.002053179545328021
epoch 550,loss 0.0011710585094988346
epoch 600,loss 0.0006679356447421014
epoch 650,loss 0.0003809648333117366
epoch 700,loss 0.00021728742285631597
epoch 750,loss 0.00012393204087857157
epoch 800,loss 7.06881401129067e-05
epoch 850,loss 4.031647404190153e-05
epoch 900,loss 2.2995525796432048e-05
epoch 950,loss 1.3116544323565904e-05
epoch 1000,loss 7.480142812710255e-06


In [130]:
# 检测模型预测结果
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[ 0.99491215],
       [ 2.995645  ],
       [ 4.996378  ],
       [ 6.997111  ],
       [ 8.997844  ],
       [10.998576  ],
       [12.99931   ],
       [15.000043  ],
       [17.000774  ],
       [19.001507  ],
       [21.00224   ]], dtype=float32)

In [131]:
# 模型的保存和读取
torch.save(model.state_dict((),"model.pkl"))

TypeError: 'tuple' object does not support item assignment